## Lat/Lon/Time Matchups the PACE Rrs Data 
### HABs Group - West Coast
* Keshav Dubedi
* Bikas Gupta
* Deborah Kutner
* Mathieu Richaud
* Dale Robinson
##### PACE Hack Week, January 2026

## Import libraries

In [1]:
import logging
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import xarray as xr
import earthaccess
import os

## Set up logging

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)
logger = logging.getLogger(__name__)
logging.getLogger("earthaccess").setLevel(logging.ERROR)

## Define functions

In [ ]:
def pace_rrs_file_finder(
    start_time: str,
    end_time: str,
    short_id: str = "PACE_OCI_L3M_RRS",
    granule_pattern: str = "*.8D.RRS.V3_1.Rrs.4km.nc",  # "*.8D.RRS.V3_1.Rrs.4km.nc"
):
    os.environ["EARTHACCESS_DISABLE_PROGRESS"] = "1"
    results = earthaccess.search_data(
        short_name=short_id,
        temporal=(start_time, end_time),
        granule_name=granule_pattern
    )
    os.environ["EARTHACCESS_DISABLE_PROGRESS"] = "1"
    return earthaccess.open(results, pqdm_kwargs={"disable": True})


def extract_mean_rrs(
    ds: xr.Dataset,
    lat: float,
    lon: float,
    location_code: str,
) -> np.ndarray:
    """
    Extract directionally averaged Rrs spectrum (all wavelengths).

    Returns:
        1D numpy array: Rrs(wavelength)
    """
    rrs = ds["Rrs"]

    lat_idx = int(np.abs(ds["lat"].values - lat).argmin())
    lon_idx = int(np.abs(ds["lon"].values - lon).argmin())

    ny = ds.sizes["lat"]
    nx = ds.sizes["lon"]

    def clip_slice(start, stop, maxval):
        return slice(max(start, 0), min(stop, maxval))

    if location_code in {"SIO", "NP"}:  # West
        lat_sel = lat_idx
        lon_sel = clip_slice(lon_idx - 8, lon_idx + 1, nx)

    elif location_code in {"CPP", "HAB_SCW", "SW", "TP"}:  # South
        lat_sel = clip_slice(lat_idx, lat_idx + 9, ny)
        lon_sel = lon_idx

    elif location_code == "MB":  # North
        lat_sel = clip_slice(lat_idx - 8, lat_idx + 1, ny)
        lon_sel = lon_idx

    else:  # Southwest
        lat_sel = clip_slice(lat_idx, lat_idx + 3, ny)
        lon_sel = clip_slice(lon_idx - 2, lon_idx + 1, nx)

    subset = rrs.isel(lat=lat_sel, lon=lon_sel)

    # Average over spatial dims that remain
    spatial_dims = [d for d in subset.dims if d in ("lat", "lon")]

    mean_rrs = subset.mean(dim=spatial_dims, skipna=True)

    return mean_rrs.values



## Set up for matchup run
* Login to earthaccess
* Load in-situ data
* Create place holder variables

In [5]:
earthaccess.login()

df_insitu = pd.read_csv("calhabs_data.csv")
df = df_insitu[(df_insitu["date"] >= "2024-03-05") & (df_insitu["date"] <= "2024-03-13")].copy()

# Placeholder — filled after first file
rrs_columns = None
rrs_data = []

## Prime the columns for results dataframce

In [12]:
first_file = None
for date in df['date'].unique():
    files = pace_rrs_file_finder(date, date)
    if files:
        first_file = files[0]
        break

if first_file:
    with xr.open_dataset(first_file) as ds:
        wavelengths = ds["wavelength"].values
        rrs_columns = [f"rrs_{int(wl)}" for wl in wavelengths]
else:
    logger.error("No PACE files found for any date. Cannot determine columns.")

## Main download and processing loop

In [16]:
# 1. Initialize rrs_columns early if possible, or handle the first-run case


for date, group in df.groupby("date"):
    logger.info("Processing date %s", date)

    files = pace_rrs_file_finder(date, date)
    
    # ISSUE 1: Appending None causes the 'NoneType' error later
    if not files:
        logger.warning("No PACE Rrs data for %s", date)
        for _ in range(len(group)):
            # If columns aren't known yet, we have to append a placeholder 
            # or handle this after the loop. 
            # If columns ARE known, use np.full(len(rrs_columns), np.nan)
            rrs_data.append([np.nan] * (len(rrs_columns) if rrs_columns else 1)) 
        continue

    with xr.open_dataset(files[0]) as ds:
        ds = ds.sortby("lat").sortby("lon")

        if rrs_columns is None:
            wavelengths = ds["wavelength"].values
            rrs_columns = [f"rrs_{int(wl)}" for wl in wavelengths]

        for _, row in group.iterrows():
            try:
                spectrum = extract_mean_rrs(ds, row["latitude"], row["longitude"], row["Location_Code"])
                # Ensure spectrum is actually a list/array, not None
                if spectrum is None:
                    rrs_data.append([np.nan] * len(rrs_columns))
                else:
                    rrs_data.append(spectrum)
            except Exception as e:
                logger.error("Failed at %s (%s): %s", row["Location_Code"], date, e)
                # ISSUE 2: If rrs_columns is None here, this will crash
                fill_val = np.full(len(rrs_columns), np.nan) if rrs_columns else [np.nan]
                rrs_data.append(fill_val)

2026-01-29 20:52:01,828 [INFO] Processing date 2024-03-06
2026-01-29 20:52:16,157 [INFO] Processing date 2024-03-11
2026-01-29 20:52:55,816 [INFO] Processing date 2024-03-13
2026-01-29 20:52:59,269 [WARNING] No PACE Rrs data for 2024-03-13


## Consolidate data and save results

In [ ]:
rrs_df = pd.DataFrame(rrs_data, columns=rrs_columns, index=df.index)

df_out = pd.concat([df.reset_index(drop=True), rrs_df], axis=1)

df_out.to_csv("cal_habs_pace_rrs_172band.csv", index=False)

logger.info("Saved Rrs output with %d bands", len(rrs_columns))